In [15]:
# pip install requests
# pip install beautifulsoup4
# pip install nltk 
# pip install spacy
# pip install textblob
# pip install wordcloud

In [16]:
import requests 
from bs4 import BeautifulSoup
import os

In [17]:
# URL des Wahlprogramms abrufen

url = 'https://www.die-linke.de/bundestagswahl-2025/wahlprogramm/'

response  = requests.get(url)
if response.status_code == 200:
     page = response.text
else:
    print(f"Seite konnte nicht abgerufen werden - Status Code {response.status_code}")

soup = BeautifulSoup(page, 'html.parser')

del page, response

In [18]:
# Liste zum Speichern der Inhalte in der richtigen Reihenfolge
content_list = []

# Alle <h4>-Elemente mit der Klasse "panel-title" finden
h4_elements = soup.find_all("h4", class_="panel-title")
last_element = h4_elements[-1]  # Letztes <h4> als Stopppunkt

# Erstes <h4>-Element suchen, um den Startpunkt zu setzen
start_element = soup.find("h4", class_="panel-title")

# Falls ein <h4> gefunden wurde, ab diesem Element iterieren
if start_element:
    # Erstes h4-Element explizit hinzufügen, falls nicht leer
    start_text = start_element.get_text(strip=True)
    if start_text:
        content_list.append(f"### {start_text}")

    # Alle nachfolgenden relevanten Elemente sammeln
    for element in start_element.find_all_next(["h3", "h4", "p", "li"]):
         # Extraktion beenden, sobald das letzte h4-Element erreicht ist (da dieses nicht Teil des Wahlprogramms ist)
        if element == last_element:
            break

        text = element.get_text(" ", strip=True)  # Leerzeichen für bessere Lesbarkeit
        
        # Nur nicht-leere Inhalte weiterverarbeiten
        if not text:
            continue
        
        # Überschriften mit ### markieren
        if element.name == "h4" and "panel-title" in element.get("class", []):
            content_list.append(f"### {text}")

        # Thesen (h3) mit ## markieren
        elif element.name == "h3":
            content_list.append(f"## {text}")

        # Alle Texte (Absätze und Listenpunkte) extrahieren
        elif element.name in ["p", "li"]:
            content_list.append(f"# {text}")

# Entferne komplett leere Einträge oder Einträge mit nur Leerzeichen (zusätzliche Absicherung)
content_list = [entry for entry in content_list if entry.strip()]

# Die extrahierten Inhalte speichern
linke_gesamt = "\n".join(content_list)
with open("linke.txt", "w", encoding="utf-8") as file:
    file.write(linke_gesamt)

# Gesamtwortanzahl berechnen
word_count = len(linke_gesamt.split())
print(f"\nGesamtanzahl der Wörter: {word_count}")


Gesamtanzahl der Wörter: 31851
